In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# 1) Veri yükleme
df = pd.read_csv("../data/raw/diabetes_012_health_indicators_BRFSS2015.csv")
df["Diabetes_binary"] = df["Diabetes_012"].apply(lambda x: 0 if x == 0 else 1)

# 2) Train-test ayırma (train.py ile aynı mantık)
df_small = df.sample(frac=0.10, random_state=42)

X = df_small.drop(["Diabetes_012", "Diabetes_binary"], axis=1)
y = df_small["Diabetes_binary"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) Eğitilmiş modeli yükle
model = joblib.load("../models/final_model.pkl")

# 4) Tahminler
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("🎯 MODEL SONUÇLARI")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob), 3))


# Model Evaluation

Bu notebook, eğitimde kullandığım final RandomForest modelini test verisi üzerinde değerlendirip metrikleri ve önemli feature'ları incelemek için yazıldı.


# Sınıflandırma Sonuçları

- Accuracy: yaklaşık **0.85**
- ROC-AUC: yaklaşık **0.80**
- Negatif sınıfta (diyabet yok) başarı oldukça yüksek.
- Pozitif sınıfta (diyabet var) **recall daha düşük**; bu da veri setindeki sınıf dengesizliği ile uyumlu.

Bu metrikler modeli bir **“erken uyarı”** aracı olarak kullanılabilir seviyede gösteriyor; ancak gerçek klinik kullanım için ek iyileştirmeler (örneğin SMOTE, threshold ayarı, farklı modeller) gerekebilir.


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Tahmin")
plt.ylabel("Gerçek")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# ROC curve
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_test, y_prob)
plt.title("ROC Curve")
plt.tight_layout()
plt.show()


# confusion Matrix & ROC Yorumu

- Confusion matrix, modelin çoğunlukla diyabet olmayan kişileri doğru tahmin ettiğini, diyabet olan grupta ise bir kısmını kaçırdığını gösteriyor.
- ROC eğrisi ve AUC ≈ 0.80, modelin rastgele tahminden belirgin şekilde daha iyi olduğunu gösteriyor.


In [ ]:
feature_imp = pd.DataFrame({
    "feature": X.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

plt.figure(figsize=(6, 5))
sns.barplot(data=feature_imp.head(10), x="importance", y="feature")
plt.title("En Önemli 10 Değişken")
plt.tight_layout()
plt.show()

feature_imp.head(20)


# Feature Importance Yorumu

En önemli değişkenler arasında:

- BMI
- Yaş kategorisi
- Genel sağlık algısı (GenHlth)
- Fiziksel sağlık günleri (PhysHlth)
- Eğitim ve gelir düzeyi

Bu sonuçlar, literatürdeki diyabet risk faktörleriyle uyumlu: yaşam tarzı, genel sağlık durumu ve sosyoekonomik faktörler önemli rol oynuyor.
